# COVID-19 di +62
<font size="3">31 Desember 2020</font>
<center><img src="https://i.postimg.cc/m2Z50Tzt/Screen-Shot-2020-12-20-at-21-17-41.png" alt="Drawing" style="width: 500px;"/></center>

## Bagian Paling Ngebosenin
<div style="text-align: justify"> Isinya cuma import library, mulung data, dan hal-hal ngebosenin lainnya. Disarankan untuk ngeskip bagian ini aja, tapi kalau mau baca yaudah gapapa. </div>

In [1]:
import json
import requests
import numpy as np
import pandas as pd
from plot_funct import *
from IPython.display import HTML

%config InlineBackend.figure_format = 'retina'

api = 'https://api.kawalcorona.com/indonesia/provinsi'
r = requests.get(api)

In [2]:
api_harian = 'https://data.covid19.go.id/public/api/update.json'
r_harian = requests.get(api_harian)

raw_data = pd.json_normalize(r_harian.json())['update.harian']
kasus_harian = pd.io.json.json_normalize(raw_data[0])

kasus_harian.rename(columns = {'key_as_string':'date',
                        'jumlah_positif_kum.value':'Terkonfirmasi',
                        'jumlah_sembuh_kum.value':'Sembuh',
                        'jumlah_meninggal_kum.value':'Meninggal',
                        'jumlah_dirawat_kum.value':'Aktif'}, 
                      inplace = True)

kasus_harian['date'] = pd.to_datetime(kasus_harian['date']).dt.date

kasus_harian.set_index('date', inplace = True)
kasus_harian[['Terkonfirmasi', 'Sembuh', 'Aktif', 'Meninggal']].tail(1)

,Terkonfirmasi,Sembuh,Aktif,Meninggal
date,,,,
2020-12-30,735124,603741,109439,21944


In [3]:
# merge BNPB's data with shp
df = pd.io.json.json_normalize(r.json())
df['attributes.Provinsi'] = df['attributes.Provinsi'].str.upper()

map_df = gpd.read_file("./Shape Files/idn_admbnda_adm1_bps_20200401.shp")
map_df['ADM1_EN'] = map_df['ADM1_EN'].str.upper()
merged = map_df.set_index('ADM1_EN').join(df.set_index('attributes.Provinsi'))

merged.rename(columns = {'attributes.Kasus_Posi':'Terkonfirmasi',
                        'attributes.Kasus_Semb':'Sembuh',
                        'attributes.Kasus_Meni':'Meninggal'}, 
              inplace = True)
merged.rename(index={'DAERAH ISTIMEWA YOGYAKARTA':'D.I. Yogyakarta'}
              ,inplace=True)

cases_list = ['Terkonfirmasi', 'Sembuh', 'Meninggal', 'Aktif']

merged['Aktif'] = merged['Terkonfirmasi'] - merged['Sembuh'] - merged['Meninggal']
merged[cases_list].head()

,Terkonfirmasi,Sembuh,Meninggal,Aktif
ADM1_EN,,,,
ACEH,7862.0,6265.0,282.0,1315.0
BALI,12631.0,11530.0,403.0,698.0
BANTEN,10831.0,8153.0,298.0,2380.0
BENGKULU,1392.0,1046.0,60.0,286.0
D.I. Yogyakarta,4511.0,3620.0,111.0,780.0


In [4]:
import matplotlib.dates as mdates
%matplotlib inline

def four_subplot(param, label, color, title):

    fig, ax = plt.subplots(2, 2); fig.set_size_inches(14,8)
    myFmt = mdates.DateFormatter('%b %Y')
    fig.suptitle(title, horizontalalignment='left', x=0.05, y=0.95,
                 verticalalignment='top', font=sf_med, fontsize = 35)
    
    for i in range(2):
        for j in range(2):
            ax[i, j].plot(kasus_harian.index, kasus_harian[param[i][j]], 
                          color = color[i][j])
            ax[i, j].set_ylabel(label[i][j])
            ax[i, j].grid('on'); 
            ax[i, j].xaxis.set_major_formatter(myFmt)
            ax[i, j].set_yticklabels(
                ['{:,}'.format(int(x)) for x in ax[i,j].get_yticks().tolist()])
            
    fig.autofmt_xdate(rotation=45)
    plt.savefig('Plot/{}.png'.format(title), dpi=300, bbox_inches='tight'); 
    plt.close()

In [5]:
label = [['Terkonfirmasi', 'Dirawat'], ['Meninggal', 'Sembuh']]
color = [['#007aff', '#ffcc00'], ['#ff3b30', '#28cd41']]

kumultatif_param = [['Terkonfirmasi', 'Aktif'], 
                ['Meninggal', 'Sembuh']]

four_subplot(kumultatif_param, label, color, 'Kasus Harian Kumultatif')

<center><img src="Plot/Kasus Harian Kumultatif.png" align="left" width="720"/></center>

In [6]:
def change_plot(var, ylab, color):
    date_format = mdates.DateFormatter('%b')
    title = 'Perubahan Harian'
    fig, ax = plt.subplots()

    fig.set_size_inches(14,8)
    fig.suptitle(title, horizontalalignment='left', x=0.066, y=0.95,
                     verticalalignment='top', font=sf_med, fontsize = 35)

    ax.plot(kasus_harian.index, kasus_harian[var], color = color)
    ax.grid('on'); ax.set_axisbelow(True);ax.set_ylabel(ylab, fontsize=15)
    ax.xaxis.set_major_formatter(date_format)

    plt.savefig('Plot/{}.png'.format(ylab), dpi=300, bbox_inches='tight'); 
    plt.close()

In [7]:
change_plot('Aktif', 'Kasus Aktif', '#ffcc00')

<center><img src="Plot/Kasus Aktif.png" align="left" width="720"/></center>

Terlihat bahwa jumlah pasien yang masih dalam perawatan masih meningkat

In [8]:
change_plot('jumlah_sembuh.value', 'Sembuh', '#28cd41')

<center><img src="Plot/Sembuh.png" align="left" width="720"/></center>

Terlihat bahwa jumlah pasien yang sembuh menunjukkan tren positif.

In [9]:
change_plot('jumlah_meninggal.value', 'Meninggal', '#ff3b30')

<center><img src="Plot/Meninggal.png" align="left" width="720"/></center>

<p align="justify">
Meskipun <i>fatality rate</i> kita di bawah 3%, namun dari grafik terlihat bahwa jumlah pasien yang meninggal dunia masih meningkat.
</p>

In [10]:
blue = make_colormap([c('#e5f1ff'), c('#83b8ff'), 0.33, c('#83b8ff'), 
                      c('#007aff'), 0.66, c('#007aff')])
terkonfirmasi_sorted = merged.sort_values(by = ['Terkonfirmasi'], ascending = True)

chloropleth(terkonfirmasi_sorted, blue, 'Terkonfirmasi', 'Terkonfirmasi'); plt.close()
top10_plot(terkonfirmasi_sorted[-10:], 'Terkonfirmasi', '#0A84FF', '10 Provinsi dengan Terkonfirmasi Tertinggi')
plt.close()

## Terkonfirmasi
<img src="Plot/Terkonfirmasi_choro.png" align="left" width="580"/>

<img src="Plot/10 Provinsi dengan Terkonfirmasi Tertinggi_bar.png" align="left" width="580"/>

In [11]:
yellow = make_colormap([c('#fff7c6'), c('#fde378'), 0.33, c('#fde378'), c('#ffcc00'), 0.66, c('#ffcc00')])
aktif_sorted = merged.sort_values(by = ['Aktif'], ascending = True)

chloropleth(aktif_sorted, yellow, 'Kasus Aktif', 'Aktif'); plt.close()
top10_plot(aktif_sorted[-10:], 'Aktif', '#ffcc00', '10 Provinsi dengan Kasus Aktif Tertinggi')
plt.close()

## Aktif
<img src="Plot/Aktif_choro.png" align="left" width="580"/>

<img src="Plot/10 Provinsi dengan Kasus Aktif Tertinggi_bar.png" align="left" width="580"/>

In [12]:
green = make_colormap([c('#c4f6cc'), c('#81e38b'), 0.33, c('#81e38b'), 
                       c('#28cd41'), 0.66, c('#28cd41')])
sembuh_sorted = merged.sort_values(by = ['Sembuh'], ascending = True)

chloropleth(sembuh_sorted, green, 'Sembuh', 'Sembuh'); plt.close()
top10_plot(sembuh_sorted[-10:], 'Sembuh', '#28cd41', 
           '10 Provinsi dengan Tingkat Kesembuhan Tertinggi')
plt.close()

## Sembuh
<img src="Plot/Sembuh_choro.png" align="left" width="580"/>

<img src="Plot/10 Provinsi dengan Tingkat Kesembuhan Tertinggi_bar.png" align="left" width="580"/>

In [13]:
red = make_colormap([c('#fddedd'), c('#ff948b'), 0.33, c('#ff948b'), 
                     c('#ff3b30'), 0.66, c('#ff3b30')])
meninggal_sorted = merged.sort_values(by = ['Meninggal'], ascending = True)

chloropleth(meninggal_sorted, red, 'Meninggal', 'Meninggal'); plt.close()
top10_plot(meninggal_sorted[-10:], 'Meninggal', '#ff3b30', 
           '10 Provinsi dengan Tingkat Kematian Tertinggi')
plt.close()

## Meninggal
<img src="Plot/Meninggal_choro.png" align="left" width="580"/>

<img src="Plot/10 Provinsi dengan Tingkat Kematian Tertinggi_bar.png" align="left" width="580"/>

## Penutup
<p align="justify">
Berdasarkan grafik perubahan harian terkonfirmasi dan kasus aktif yang masih terus meningkat maka dapat disimpulkan bahwa pandemi ini belum berakhir hinggal 2021 mendatang. Semoga di tahun yang akan datang kita semua dapat mengatasi pandemi ini dengan lebih baik. <br/> 
</p>
<p align="justify">
Pake masker, <i>stay safe</i>, dan jangan lupa bahagia.<br/> 
</p>
<p align="justify">
Selamat tahun baru 2021.
</p>